In [ ]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

rc('text', usetex=True)
font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 14}
rc('font', **font)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
model_vals = []
solver_vals = []
loss_vals = []
overall_scores_rows = []
columns = None

for results_file in pathlib.Path('./eval').rglob('eval_results.csv'):
    model_dir = results_file.parent.parent
    solver_dir = model_dir.parent
    loss_dir = solver_dir.parent

    model = model_dir.stem
    solver = solver_dir.stem
    loss = loss_dir.stem

    model_vals.append(model)
    solver_vals.append(solver)
    loss_vals.append(loss)

    df = pd.read_csv(str(results_file), index_col=0)
    columns = df.columns
    overall_scores = df.iloc[-1]
    overall_scores_rows.append(overall_scores)

df_orig = pd.DataFrame(overall_scores_rows, columns=columns)
df_orig.insert(0, 'model', model_vals)
df_orig.insert(1, 'solver', solver_vals)
df_orig.insert(2, 'loss', loss_vals)
df_orig.reset_index(drop=True, inplace=True)
df_orig

In [ ]:
df_orig_sub = df_orig[df_orig['model'] == '0040000']
df_orig_sub

In [ ]:
df = df_orig[
    (df_orig['model'] == '0030000') |
    (df_orig['model'] == '0040000') |
    (df_orig['model'] == '0050000')
]
df

In [ ]:
def plot_tracker_loss_solver_comparison(
    df, x_col, y_col, x_label, y_label, x_units='\%', y_units='\%'
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label

    fig, ax = plt.subplots(figsize=(20, 12), nrows=1, ncols=1)

    base_colors =  list(mcolors.BASE_COLORS.keys())
    unique_models = df['model'].unique().tolist()

    size = 35 ** 2
    for solver, group_solver_df in df.groupby(['solver']):
        if solver == 'orig':
            facecolors = 'none'
        else:
            facecolors = edgecolors = None
        
        for loss, group_loss_df in group_solver_df.groupby(['loss']):
            if loss == 'none':
                marker = 'o'
            elif loss == 'contr':
                marker = 'X'
            else:  # triplet
                marker = '^'
        
            for model, group_model_df in group_loss_df.groupby(['model']):
                xs, ys = group_model_df[x_col], group_model_df[y_col]    
                
                curr_color = base_colors[unique_models.index(model)]
                label = rf'${solver}, {loss}, {str(int(model))}$'
                if facecolors == 'none':
                    color = None
                    edgecolors = curr_color
                else:
                    color = curr_color
                
                ax.scatter(
                    xs, ys, s=size, c=color, label=label, marker=marker,
                    facecolors=facecolors, edgecolors=edgecolors, linewidth=4,
                    alpha=0.7
                )
    
    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_title(
        r'$\textbf{Tracker Performance Comparison --- ' + 
        rf'Training Iterations, Loss, Solver --- {x_label} vs. {y_label}}}$'
    )
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='best', markerscale=0.5, fontsize=14)

    return fig

fig_mota_motp = plot_tracker_loss_solver_comparison(
    df, 'mota', 'motp', 'MOTA', 'MOTP'
)
fig_mota_motp.savefig('tracker_cmp_plot_mota_motp.png', dpi=300)

In [ ]:
fig_rec_prec = plot_tracker_loss_solver_comparison(
    df, 'recall', 'precision', 'Recall', 'Precision'
)
fig_rec_prec.savefig('tracker_cmp_plot_rec_prec.png', dpi=300)